In [1]:
import os
os.chdir('..')
from src.model.HMM.HMM import HMM
from src.model.HMM.data import train_test_split, get_dataset

In [2]:
exe_path = 'data/processed/exe.csv'
X, y = get_dataset(exe_path)
X_train, y_train, X_test, y_test = train_test_split(X, y)
model = HMM()
model.fit(X_train, y_train)

### Top-1 accuracy

In [3]:
pred_1 = model.predict(X_test)
accuracy_1 = model.evaluate(pred_1, y_test)
accuracy_1

0.474059003051882

### Top-2 accuracy

In [4]:
pred_2 = model.predict(X_test, top=2)
accuracy_2 = model.evaluate(pred_2, y_test)
accuracy_2

0.6581892166836215

### Top-3 accuracy

In [5]:
pred_3 = model.predict(X_test, top=3)
accuracy_3 = model.evaluate(pred_3, y_test)
accuracy_3

0.7589013224821973